In [1]:
import pandas as pd
import json
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist
import string

In [2]:
user_demo_profiles_path = './data/User demo profiles.json'
labeled_tweets_path = './data/Twitter_User_Handles_labeled_tweets.json'
labeled_users = './data/labeled_users.csv'

df_tweets = pd.read_json(labeled_tweets_path, orient='index')
df_profiles = pd.read_json(user_demo_profiles_path, orient='values')

In [3]:
df_profiles

,id,name,screen_name,description,lang,img_path
0,12488,Chad Boyce,djsnipa1,"Multimedia Developer, Graphic Designer, DJ, an...",NA,profile pics/60147.jpeg
1,719703,Tomato 😷,Tomato,🇭🇰Rise Up!,NA,profile pics/60148.jpeg
2,722153,blank,blank,"Someone who thinks too much, acts too little.",NA,profile pics/60149.jpeg
3,749003,A ring made from a spoon --- E83.31,IYQbd,Has the kind of luck that turns a professional...,NA,profile pics/60150.jpeg
4,1896301728,عبدالله,755531,,NA,profile pics/60151.jpeg
...,...,...,...,...,...,...
3604,4895390642,Dalton's Eyeliner,FlyAwayEyeliner,"I'm Dalton's eyeliner, who are you? Are you Da...",NA,profile pics/64267.jpeg
3605,4895831833,Jamie Loftus,cowboyjamie77,"41 year old male. interests include learning, ...",NA,profile pics/64268.jpeg
3606,4921995243,saved,memelady,,NA,profile pics/64269.jpeg
3607,4924158634,Winter da CoffeeCat,WinterStar21,"Big fan of cats, coffee, horror stuff and anim...",NA,profile pics/64270.jpeg


In [4]:
df_tweets.index.name = 'user_id'

In [5]:
df_tweets

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
12488,"YKAR, a futuristic sans serif font by @Emmeran...",@MBonvoyAssist Who can I contact about the ver...,@SSlnes I’d like to win!,@LuckyDucksNFT @Rydog,Now I'm heading to B1000th Floor! #quickrogue,Check this out! https://t.co/eOtMJ99jE7,@AskLyft Thank you! Yes I'm doing it now,@AskLyft Please help me! I got scammed and nee...,@catalogscanner #AnimalCrossing #ACNH #Nintend...,@catalogscanner #AnimalCrossing #ACNH #Nintend...,...,@BandaruOrganics Hello! Will you get these in ...,Evernote helps me remember everything. Get it ...,FINAL FANTASY ALL THE BRAVEST is here! An easy...,My MIR1 flying from San Francisco to Phoenix i...,@MikeRogers0 I love LivePage! Thank you so muc...,I just signed up for @CopyApp and got 5 GB fre...,Say hello to the future of e-mail at http://t....,Check out my personalized 2012 Gaming Summary ...,Mark Bale feat. Cosmo Klein - Gimme Some Love ...,Matty Menck - Rain (Original Mix) http://t.co/...
719703,"In other words, it’s good news about the vacci...",People forget that Trump didn’t even want to s...,The Taiwan government’s response to COVID is s...,#ACNH https://t.co/cEUo84rfvV,Quite happy with yesterday's work on my zen ga...,"Yikes. It isn’t easy to comprehend, as with al...",It’s harder and harder to get excited about bu...,Excellent news. I’ve been feeling so exhausted...,What a line. https://t.co/CnWN9lx4uS,"As a HKer, I’ve gotten used to disappointments...",...,This #iPhone6′s vibration is weak. I got a cal...,@stewart Young people in Hong Kong (myself inc...,Now I really have to go to New York in the nex...,"Wow, congrats @StephenAtHome. Very mixed feeli...",Glitch was a wonderful game and I really miss ...,I was in the game until the final few minutes....,"So, bringing my MacBook, I went to the hotel l...",But the flight was majorly delayed! Thank God ...,I was going to miss the last hours of the game...,Tonight I'm getting pretty emotional when I th...
749003,would it be fair to call lil nas x the first s...,@BillyKumo He did,Decided to only engage with Batman and Batman ...,I'm reading a 1953 Mad magazine Batman parody ...,"Happy Batman Day, Batman people!\nBatman!",Because it's not constrained by the suits the ...,In the second half Robbie declares that his ca...,iffy https://t.co/GnfMHs3h06,I forgot the second wink which makes the first...,"Spike hires a campaign manager, Olio Slick (😉)...",...,ha canadian mathen. i get it https://t.co/crx2...,@madmup https://t.co/XvnNzVcw5v,This whole run feels like it wants to be that ...,This page layout makes no sense to me.\n\nI wa...,There are 911 stans https://t.co/BnRXVrsbb6,"I triggered the ""do you really want to tweet t...",@MnDPS_DPS @MnDPS_MSP Get f.ucked,This is dystopic https://t.co/QreOku3Zit,This is neat. The motion lines show the monste...,Love\nAngel\nMovie\nBaby https://t.co/L2efpaqKrh
822540,@Asmongold ❤️ 🙏,@Hunter4J @Wario64 bonk,@FF_XIV_EN #FFXIV400kSweepstakes,@DaysUntilEW Nice,@AndrewMFanco @jdflynn Ed McBoy,@Wario64 OOS. getting real tired of GameStop t...,"@Wario64 XSX is sold out, was removed from car...","@Wario64 I have pro, can't add to cart",@pixelpipes @dark1x Wish they had used that al...,@WBNG12News RIP Squishy ♥️,...,@kazoodac @Wario64 Stacking 20% off with $10 c...,@Patriotssuck19 @NYRangers lol,@nealmchale @RickCarpiniello Tarasenko or Jade...,@RickCarpiniello The tradition of Festivus beg...,"@Wario64 Glad I spotted this, finally opened m...","@RickCarpiniello You too, Carp.",@MoadhtheGreat @Wario64 I was shocked at how n...,@Wario64 I remember when Newegg was known for ...,@djdresden Yub Nub,@BinghamtonNow What happened to staying 6 feet...
865071,@robo_james How about pizza dipped in water 🤦🏻‍♂️,Day 21 of Quarantine: We installed a trapeze i...,"I love Portuguese egg tarts too, but is this r...","Well, that was an unexpected memory from the r...","The #GDPR has many benefits, but needing to co...",Why does nobody talk about the f

In [6]:
df_users = pd.read_csv(labeled_users)

In [7]:
df_users.head()

,user_id,is_female,year_born,race
0,12488.0,0.0,1980.0,4.0
1,719703.0,0.0,1985.0,4.0
2,722153.0,1.0,1973.0,3.0
3,749003.0,0.0,1982.0,5.0
4,755531.0,0.0,1982.0,4.0


### Preprocssing Text
- all lowercase
- remove urls
- remove @user
- remove #hashtag
- remove punctuations 
- split dataset to train and validation

In [8]:
# A function to preprocessing a column of tweets, the input should be df_tweets['i'], i = 0,...,99
def cleaned_tweet(tweets):
    #lower case
    tweets = tweets.str.lower()
    #remove urls
    tweets = tweets.apply(lambda x: re.sub(r'https?:\/\/\S+', '', str(x)))
    # remove url/website that didn't use http, is only checking for .com websites 
    # so words that are seperated by a . are not removed
    tweets = tweets.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
    # remove @user
    tweets = tweets.apply(lambda x: re.sub(r'@[A-Za-z0-9_]+', '', x))
    # remove #hashtag
    tweets = tweets.apply(lambda x: re.sub(r'#[A-Za-z0-9_]+', '', x))
    # remove special characters, numbers, punctuations
    tweets = tweets.apply(lambda x: re.sub(r'\W+', ' ', x))
    # remove short words length less than 3
    tweets = tweets.apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
    # remove Re-Tweet
    tweets = tweets.apply(lambda x: re.sub(r'^RT[\s]+', '', x))
    return tweets

In [ ]:
for col in df_tweets.columns:
    df_tweets[col] = cleaned_tweet(df_tweets[col])
df_tweets

In [ ]:
# Combine the race and age label to the preprocessed tweets
df_cleanedtweets = pd.merge(df_tweets, df_users, on="user_id")
df_cleanedtweets

In [ ]:
# get the age label through year_born column
df_cleanedtweets['age'] = df_cleanedtweets['year_born'].apply(lambda x: 2021-x)
df_cleanedtweets['age'] = df_cleanedtweets['age'].apply(lambda x: 1 if x >= 21 else 0)
df_cleanedtweets

In [ ]:
# df_cleanedtweets.to_csv('./data/preprocessed_tweets_with_race&age.csv')

In [ ]:
df_cleanedtweets['age'].value_counts()

#### Merge all 100 tweets into one column

In [ ]:
df_cleanedtweets['all_tweets'] = df_cleanedtweets[[i for i in range(100)]].agg(' '.join, axis=1)

In [ ]:
df_cleanedtweets['all_tweets']

In [ ]:
assert len(df_cleanedtweets[0]) == len(df_cleanedtweets['all_tweets'])

#### remove None in `all_tweets`

In [ ]:
import re

tryon = df_cleanedtweets['all_tweets'][3271]
delted = re.sub('None', '',tryon)

str_ls = []
for s in df_cleanedtweets['all_tweets']:
    temp = re.sub('None', '', s)
    str_ls.append(temp)
    
df_cleanedtweets['all_tweets'] = str_ls

In [ ]:
df_cleanedtweets['all_tweets']

Noted that 3271 is empty string, so remove empty string after clean "None" in `all_tweets` column

In [ ]:
import numpy as np

df_cleanedtweets['all_tweets'] = df_cleanedtweets['all_tweets'].replace(r'^\s*$', np.NaN, regex=True)
df_cleanedtweets['all_tweets']

In [ ]:
df_cleanedtweets['all_tweets'].isna().sum()

In [ ]:
df_cleanedtweets.dropna(subset=['all_tweets'], inplace=True)

In [ ]:
df_cleanedtweets

In [ ]:
col = ['user_id', 'all_tweets', 'is_female', 'year_born', 'race', 'age']

In [ ]:
df_cleanedtweets = df_cleanedtweets[col]

In [ ]:
df_cleanedtweets

In [ ]:
df_cleanedtweets.to_csv('./data/preprocessed_tweets_with_race&age.csv')